## **Data Cleaning Steps Overview**

 1. **Read the raw file** using the correct delimiter (`;`).
 2. **Remove empty columns and rows** to handle any trailing semicolons r blank lines.
 3. **Save the cleaned data** to the `data/raw/` folder.
 4. **Preview the results** by printing the shape and the first few rows of the cleaned dataset.

In [2]:

import pandas as pd

# Path to the raw dataset
raw_path = 'C:/Users/HP/iCloudDrive/school/DSA 2040/Project/DSA2040A_DataMining_group1/OSMI Mental Health Tech Survey 2014 Dataset.csv'
cleaned_path = '../data/raw/OSMI_Mental_Health_Cleaned.csv'

# 1. Read the raw CSV with the correct delimiter
df = pd.read_csv(raw_path, delimiter=';', dtype=str)

# 2. Remove any columns that are completely empty (sometimes caused by trailing delimiters)
df = df.dropna(axis=1, how='all')

# 3. Remove any rows that are completely empty
df = df.dropna(axis=0, how='all')

# 4. Save the cleaned file (still semicolon-delimited for now)
df.to_csv(cleaned_path, index=False, sep=';')

# 5. Preview the data
print("Shape:", df.shape)
print(df.head())

Shape: (1259, 27)
          Timestamp Age  Gender         Country state self_employed  \
0  27/08/2014 11:29  37  Female   United States    IL           NaN   
1  27/08/2014 11:29  44       M   United States    IN           NaN   
2  27/08/2014 11:29  32    Male          Canada   NaN           NaN   
3  27/08/2014 11:29  31    Male  United Kingdom   NaN           NaN   
4  27/08/2014 11:30  31    Male   United States    TX           NaN   

  family_history treatment work_interfere    no_employees  ...  \
0             No       Yes          Often          WTD-25  ...   
1             No        No         Rarely  More than 1000  ...   
2             No        No         Rarely          WTD-25  ...   
3            Yes       Yes          Often          26-100  ...   
4             No        No          Never         100-500  ...   

                leave mental_health_consequence phys_health_consequence  \
0       Somewhat easy                        No                      No   
1       

 ## 2. Data Transformation
 In this step, we will perform the following data transformation tasks:
  - Standardize column names (e.g., make lowercase, replace spaces with underscores)
- Normalize categorical values (e.g., gender, country)
- Handle missing values appropriately
- Convert data types where necessary (e.g., age to integer)
- Create any new features if needed for analysis
- Save the transformed data for further analysis

## 2.1 Standardize Missing Values

In [3]:

import numpy as np

# 1. Define all known missing value indicators
missing_vals = ['NA', 'N/A', 'n/a', 'na', 'Not sure', "Don't know", '', ' ']

# 2. Replace all such values with np.nan across the entire DataFrame
df_clean = df.replace(missing_vals, np.nan)

# 3. View missing value counts per column
print("Missing values per column:")
print(df_clean.isnull().sum())

# 4. View total missing values in the dataset
print(f"\nTotal missing values in dataset: {df_clean.isnull().sum().sum()}")

# 5. (Optional) View percentage of missing values per column
print("\nPercentage of missing values per column:")
print((df_clean.isnull().mean() * 100).round(2))

# 6. Show a sample of rows with missing data
print("\nSample rows with missing data:")
print(df_clean[df_clean.isnull().any(axis=1)].head())

Missing values per column:
Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_employees                    0
remote_work                     0
tech_company                    0
benefits                      408
care_options                  314
wellness_program              188
seek_help                     363
anonymity                     819
leave                         563
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical            576
obs_consequence                 0
comments                     1096
dtype: int64

Total missing values in dataset: 5124

Pe

# # 2.2 Here we clean the 'Age' column:
 Convert the 'Age' column to numeric values, setting any invalid entries to NaN.
- Remove rows where age is outside the range 15 to 80.
- Count and report how many rows were removed due to implausible ages.
- Check for duplicate rows, remove them if found, and report how many were removed.
- Finally, display information and a preview of the cleaned DataFrame.


In [4]:

# 1. Convert Age to numeric, coerce errors to NaN
df_clean['Age'] = pd.to_numeric(df_clean['Age'], errors='coerce')

# 2. Remove age outliers (keep ages between 15 and 80)
before = len(df_clean)
df_clean = df_clean[(df_clean['Age'] >= 15) & (df_clean['Age'] <= 80)]
after = len(df_clean)
print(f"Removed {before - after} rows with implausible ages.")

# 3. (Optional) Check for duplicates and remove them
dupes = df_clean.duplicated().sum()
if dupes > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"Removed {dupes} duplicate rows.")

# 4. Preview the cleaned data
print(df_clean.info())
print(df_clean.head())

Removed 8 rows with implausible ages.
Removed 1 duplicate rows.
<class 'pandas.core.frame.DataFrame'>
Index: 1250 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1250 non-null   object
 1   Age                        1250 non-null   int64 
 2   Gender                     1250 non-null   object
 3   Country                    1250 non-null   object
 4   state                      738 non-null    object
 5   self_employed              1232 non-null   object
 6   family_history             1250 non-null   object
 7   treatment                  1250 non-null   object
 8   work_interfere             988 non-null    object
 9   no_employees               1250 non-null   object
 10  remote_work                1250 non-null   object
 11  tech_company               1250 non-null   object
 12  benefits                   843 non-null    object
 13  care

## 2.3 Standardizing Gender Values

In this section, we clean and standardize the 'Gender' column in our dataset. The original data contains many variations and misspellings for gender entries. To address this:

1. We define a dictionary called `gender_map` that maps various possible gender responses to three categories: 'male', 'female', or 'other'. For example, entries like 'm', 'male-ish', and 'cis man' are all mapped to 'male', while 'f', 'woman', and 'cis-female/femme' are mapped to 'female'. Any entries that refer to transgender, non-binary, or other less common gender identities are mapped to 'other'.

2. We then standardize the 'Gender' column by:
   - Converting all values to strings,
   - Stripping whitespace,
   - Converting to lowercase,
   - Mapping each value using our `gender_map` dictionary.
   - Any value not found in the mapping is set to 'other' by default.

3. We print the unique values in the cleaned 'Gender' column to verify the standardization.

4. Finally, we save the cleaned DataFrame to a CSV file in the transformed data folder for future use.


In [5]:

# 1. Define a mapping for gender values
gender_map = {
    'male': 'male', 'm': 'male', 'male-ish': 'male', 'maile': 'male', 'mal': 'male', 'cis male': 'male', 'cis man': 'male',
    'female': 'female', 'f': 'female', 'woman': 'female', 'cis female': 'female', 'cis-female/femme': 'female', 'femail': 'female',
    'trans-female': 'other', 'trans woman': 'other', 'trans female': 'other', 'trans male': 'other', 'trans man': 'other',
    'genderqueer': 'other', 'non-binary': 'other', 'agender': 'other', 'androgyne': 'other', 'other': 'other',
    'fluid': 'other', 'queer': 'other', 'all': 'other', 'enby': 'other', 'p': 'other', 'a little about you': 'other',
    # Add more mappings as needed
}

# 2. Standardize the Gender column
df_clean['Gender'] = (
    df_clean['Gender']
    .astype(str)
    .str.strip()
    .str.lower()
    .map(gender_map)
    .fillna('other')
)

# 3. Preview the cleaned Gender values
print("Unique values in Gender after cleaning:", df_clean['Gender'].unique())

# 4. Save the cleaned DataFrame to the transformed data folder
output_path = '../data/transformed/OSMI_Mental_Health_Cleaned.csv'
df_clean.to_csv(output_path, index=False)
print(f"Cleaned data saved to {output_path}")

Unique values in Gender after cleaning: ['female' 'male' 'other']
Cleaned data saved to ../data/transformed/OSMI_Mental_Health_Cleaned.csv


# 2.4 Standardizing Categorical Columns
This section cleans up several categorical columns in the dataset. 
First, it defines a list of columns that contain categorical responses (like 'yes', 'no', etc.).
For each of these columns, it converts all values to lowercase and removes any leading or trailing spaces, 
which helps ensure consistency (e.g., ' Yes ' becomes 'yes').

Next, for columns that are expected to be binary (i.e., only 'yes' or 'no'), 
it maps any value that matches 'yes' or 'no' to itself, and leaves other values unchanged. 
This step helps to further standardize the data, making it easier to analyze later.

In [7]:

# List of columns to standardize
cat_cols = [
    'self_employed', 'family_history', 'treatment', 'remote_work', 'tech_company',
    'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity',
    'leave', 'mental_health_consequence', 'phys_health_consequence',
    'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
    'mental_vs_physical', 'obs_consequence'
]

# Standardize to lowercase and strip whitespace
for col in cat_cols:
    df_clean[col] = df_clean[col].astype(str).str.strip().str.lower().replace('nan', np.nan)

# Optionally, map to only 'yes', 'no', or np.nan for binary columns
binary_map = {'yes': 'yes', 'no': 'no'}
for col in cat_cols:
    df_clean[col] = df_clean[col].map(binary_map).fillna(df_clean[col])

# 2.5 Handling Missing Values in Categorical Columns
This section deals with missing values in the dataset, focusing on categorical columns.

1. For a set of binary/categorical columns (like 'self_employed', 'family_history', etc.), 
   the code checks the percentage of missing values. If less than 10% of the values are missing in a column,
   it fills those missing values with the most common value (the mode) for that column. 
   If more than 10% are missing, it leaves them as missing for now and prints a message.

2. For the 'state' column, any missing values are filled with the string 'Not US', 
   indicating that the respondent is not from the United States or did not provide a state.

3. The code then checks all columns and drops any column where more than 50% of the values are missing, 
   as these columns are likely not useful for analysis.

4. Finally, it prints a summary showing how many missing values remain in each column after these steps.

In [8]:

# 1. Impute binary/categorical columns with mode if only a few values are missing
cat_cols = ['self_employed', 'family_history', 'treatment', 'remote_work', 'tech_company',
            'benefits', 'care_options', 'wellness_program', 'seek_help']

for col in cat_cols:
    missing_count = df_clean[col].isnull().sum()
    total_count = len(df_clean)
    missing_pct = missing_count / total_count
    # If less than 10% missing, fill with mode
    if missing_pct < 0.1:
        mode_val = df_clean[col].mode(dropna=True)
        if not mode_val.empty:
            df_clean[col] = df_clean[col].fillna(mode_val[0])
            print(f"Filled missing values in '{col}' with mode: {mode_val[0]}")
    else:
        print(f"Column '{col}' has {missing_pct:.1%} missing; left as NaN for now.")

# 2. For 'state', fill missing with 'Not US'
df_clean['state'] = df_clean['state'].fillna('Not US')
print("Filled missing 'state' with 'Not US'.")

# 3. Drop columns with too many missing values (e.g., >50% missing)
threshold = 0.5
cols_to_drop = [col for col in df_clean.columns if df_clean[col].isnull().mean() > threshold]
df_clean = df_clean.drop(columns=cols_to_drop)
print(f"Dropped columns with >50% missing values: {cols_to_drop}")

# 4. Show missing value summary after imputation
print("\nMissing values per column after imputation and dropping:")
print(df_clean.isnull().sum())

Filled missing values in 'self_employed' with mode: no
Filled missing values in 'family_history' with mode: no
Filled missing values in 'treatment' with mode: yes
Filled missing values in 'remote_work' with mode: no
Filled missing values in 'tech_company' with mode: yes
Column 'benefits' has 32.6% missing; left as NaN for now.
Column 'care_options' has 25.0% missing; left as NaN for now.
Column 'wellness_program' has 15.0% missing; left as NaN for now.
Column 'seek_help' has 29.0% missing; left as NaN for now.
Filled missing 'state' with 'Not US'.
Dropped columns with >50% missing values: ['anonymity', 'comments']

Missing values per column after imputation and dropping:
Timestamp                      0
Age                            0
Gender                         0
Country                        0
state                          0
self_employed                  0
family_history                 0
treatment                      0
work_interfere               262
no_employees           

# 2.6 imputation of missing values
This section handles further imputation of missing values. It first fills missing values in columns with moderate missingness ('work_interfere', 'benefits', 'care_options', 'wellness_program', 'seek_help') using the mode of each column. Then, for columns with high missingness ('leave', 'mental_vs_physical'), it fills missing values with the string 'unknown'. Finally, it prints a summary of the remaining missing values in each column after these imputations.

In [ ]:


# 1. Columns with moderate missingness: impute with mode
moderate_missing_cols = ['work_interfere', 'benefits', 'care_options', 'wellness_program', 'seek_help']
for col in moderate_missing_cols:
    mode_val = df_clean[col].mode(dropna=True)
    if not mode_val.empty:
        df_clean[col] = df_clean[col].fillna(mode_val[0])
        print(f"Filled missing values in '{col}' with mode: {mode_val[0]}")

# 2. Columns with high missingness: fill with 'unknown'
high_missing_cols = ['leave', 'mental_vs_physical']
for col in high_missing_cols:
    df_clean[col] = df_clean[col].fillna('unknown')
    print(f"Filled missing values in '{col}' with 'unknown'.")

# 3. Show missing value summary after final imputation
print("\nMissing values per column after final imputation:")
print(df_clean.isnull().sum())

Filled missing values in 'work_interfere' with mode: Sometimes
Filled missing values in 'benefits' with mode: yes
Filled missing values in 'care_options' with mode: no
Filled missing values in 'wellness_program' with mode: no
Filled missing values in 'seek_help' with mode: no
Filled missing values in 'leave' with 'unknown'.
Filled missing values in 'mental_vs_physical' with 'unknown'.

Missing values per column after final imputation:
Timestamp                    0
Age                          0
Gender                       0
Country                      0
state                        0
self_employed                0
family_history               0
treatment                    0
work_interfere               0
no_employees                 0
remote_work                  0
tech_company                 0
benefits                     0
care_options                 0
wellness_program             0
seek_help                    0
leave                        0
mental_health_consequence    0
phy

# 3:  Feature Engineering

In [11]:

bins = [0, 24, 34, 44, 54, 100]
labels = ['<25', '25-34', '35-44', '45-54', '55+']
df_clean['age_group'] = pd.cut(df_clean['Age'], bins=bins, labels=labels)

print(df_clean[['Age', 'age_group']].head())

   Age age_group
0   37     35-44
1   44     35-44
2   32     25-34
3   31     25-34
4   31     25-34


In [14]:

# Define the output path 
output_path = '../data/transformed/OSMI_Mental_Health_Final.csv'

# Save the DataFrame
df_clean.to_csv(output_path, index=False)

print(f"Final cleaned data saved to {output_path}")

Final cleaned data saved to ../data/transformed/OSMI_Mental_Health_Final.csv


In [ ]:

import pandas as pd

# Path to the cleaned data (adjust if needed)
cleaned_path = '../data/transformed/OSMI_Mental_Health_Final.csv'

# Load the cleaned data
df_final = pd.read_csv(cleaned_path)

# Show the shape and first few rows
print("Shape of the final cleaned dataset:", df_final.shape)
print(df_final.head())

# (Optional) Show column names
print("\nColumns in the final cleaned dataset:")
print(df_final.columns.tolist())

Shape of the final cleaned dataset: (1250, 26)
          Timestamp  Age  Gender         Country   state self_employed  \
0  27/08/2014 11:29   37  female   United States      IL            no   
1  27/08/2014 11:29   44    male   United States      IN            no   
2  27/08/2014 11:29   32    male          Canada  Not US            no   
3  27/08/2014 11:29   31    male  United Kingdom  Not US            no   
4  27/08/2014 11:30   31    male   United States      TX            no   

  family_history treatment work_interfere    no_employees  ...  \
0             no       yes          Often          WTD-25  ...   
1             no        no         Rarely  More than 1000  ...   
2             no        no         Rarely          WTD-25  ...   
3            yes       yes          Often          26-100  ...   
4             no        no          Never         100-500  ...   

                leave mental_health_consequence phys_health_consequence  \
0       somewhat easy               